# Quante persone saranno contagiate?

Si vedono tante proiezioni sui contagiati, tutte fanno vedere una curva esponenziale (che shizza verso l'alto).
Questo effetto piuttosto drammatico è dato più dalla scelta del modello statistico che dai dati in sè.

In questo notebook confrontiamo il modello esponenziale con un modello più sobrio, che tiene conto del più comune andamento dei fenomeni biologici: cioè una rapida crescita seguita da un assestamento. Faremo anche una predizione su quando avverrà questo assestamento.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt